### Implementação do trabalho do aluno que obteve o melhor conjunto de parâmetros e hiperparêmtros para o problema

In [1]:
import time
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import LeakyReLU, PReLU
from keras.layers.core import Activation, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D

Using TensorFlow backend.


In [2]:
batch_size = 32
learning_rate = 5e-4
decay = 0.0
momentum = 0.9
num_classes = 10
epochs = 20

In [4]:
(x_train_aux, y_train_aux), (x_test_aux, y_test_aux) = mnist.load_data()

#training - 90%
x_train = np.concatenate((x_train_aux, x_test_aux[:3000]))
y_train = np.concatenate((y_train_aux, y_test_aux[:3000]))

#validation - 5%
x_validate = x_test_aux[3000:6500]
y_validate = y_test_aux[3000:6500]

#test - 5%
x_test = x_test_aux[6500:]
y_test = y_test_aux[6500:]

#Reshape from 28x28 to 32x32
x_train = np.array(x_train)
x_validate = np.array(x_validate)
x_test = np.array(x_test)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_validate = x_validate.reshape(x_validate.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_validate = np.pad(x_validate, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

x_train = x_train.astype('float32')
x_validate = x_validate.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_validate /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_validate = to_categorical(y_validate, num_classes)
y_test = to_categorical(y_test, num_classes)


print("shape x_train: ", x_train.shape)
print("shape y_train: ", y_train.shape)

print("shape x_validate: ", x_validate.shape)
print("shape y_validate: ", y_validate.shape)

print("shape x_test: ", x_test.shape)
print("shape y_test: ", y_test.shape)

shape x_train:  (63000, 32, 32, 1)
shape y_train:  (63000, 10)
shape x_validate:  (3500, 32, 32, 1)
shape y_validate:  (3500, 10)
shape x_test:  (3500, 32, 32, 1)
shape y_test:  (3500, 10)


In [5]:
model = Sequential()

#Layer 1 - Conv 2D
model.add(Convolution2D(filters = 6, 
                 kernel_size = 3, 
                 strides = 1,
                 kernel_initializer='he_normal',
                 activation = 'relu', 
                 input_shape = (32, 32, 1)))

#Pooling 2x2
model.add(MaxPooling2D(pool_size = 2, strides = 2))

#Layer 2 - Conv 2D
model.add(Convolution2D(filters = 16, 
                 kernel_size = 3, 
                 strides = 1, 
                 kernel_initializer='he_normal',
                 activation = 'relu', 
                 input_shape = (15, 15, 6)))

#Layer 3 - Conv 2D
model.add(Convolution2D(filters = 20, 
                 kernel_size = 3, 
                 strides = 1, 
                 kernel_initializer='he_normal',
                 activation = 'relu', 
                 input_shape = (13, 13, 6)))

#Pooling 2x2
model.add(MaxPooling2D(pool_size = 2, strides = 2))

model.add(Flatten())

#Layer 4 - Full Connection
model.add(Dense(units = 160, kernel_initializer='he_normal', activation = 'relu'))

#Layer 5 - Full Connection
model.add(Dense(units = 100, kernel_initializer='he_normal', activation = 'relu'))

#Layer 6 - Full Connection
model.add(Dense(units = 84, kernel_initializer='he_normal', activation = 'relu'))

#Layer 7 - Output Layer
model.add(Dense(units = num_classes, kernel_initializer='he_normal', activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 6)         60        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 20)        2900      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 160)               80160     
__________

In [6]:
adam = optimizers.Adam(lr = learning_rate, decay = decay)

model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['accuracy'])
start = time.time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data = (x_validate, y_validate))  
end = time.time()
print("Tempo de treinamento: {} s".format(end - start))

Train on 63000 samples, validate on 3500 samples
Epoch 1/20
63000/63000 [==============================] - 62s 992us/step - loss: 0.2035 - acc: 0.9371 - val_loss: 0.0952 - val_acc: 0.9691
Epoch 2/20
63000/63000 [==============================] - 54s 862us/step - loss: 0.0624 - acc: 0.9808 - val_loss: 0.0383 - val_acc: 0.9857
Epoch 3/20
63000/63000 [==============================] - 56s 896us/step - loss: 0.0443 - acc: 0.9858 - val_loss: 0.0325 - val_acc: 0.9906
Epoch 4/20
63000/63000 [==============================] - 55s 877us/step - loss: 0.0344 - acc: 0.9895 - val_loss: 0.0300 - val_acc: 0.9891
Epoch 5/20
63000/63000 [==============================] - 56s 881us/step - loss: 0.0270 - acc: 0.9913 - val_loss: 0.0229 - val_acc: 0.9920
Epoch 6/20
63000/63000 [==============================] - 58s 917us/step - loss: 0.0220 - acc: 0.9930 - val_loss: 0.0294 - val_acc: 0.9923
Epoch 7/20
63000/63000 [==============================] - 56s 888us/step - loss: 0.0178 - acc: 0.9943 - val_loss: 0.0

In [7]:
#Test
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Loss: {}  Acc: {}'.format(score[0], score[1]))

3500/3500 [==============================] - 1s 361us/step
Loss: 0.03227764595223666  Acc: 0.992
